In [8]:
%reload_ext autoreload
%autoreload 2

In [9]:
import os

import pandas as pd
import requests_cache
from loguru import logger
from pyprojroot import here
from tqdm.notebook import tqdm

requests_cache.install_cache(str(here()))

os.chdir(here())

In [10]:
from municipal_performance_scraping.transformers import (
    transform_performance_for_evaluations,
)

In [11]:
from municipal_performance_scraping.extractors import extract_performance_data
from municipal_performance_scraping.loaders import load_evaluation
from municipal_performance_scraping.transformers import (
    transform_performance_for_evaluations,
)

In [14]:
from municipal_performance_scraping.models import engine

In [26]:
ids = pd.read_sql("select * from municipalities", con=engine)[
    "municipality_id"
].to_list()

In [16]:
import concurrent.futures

In [44]:
transformed = []
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_mids = {executor.submit(extract_performance_data, mid, 2017): mid for mid in ids}
    errors = []
    for future in tqdm(concurrent.futures.as_completed(future_to_mids), total=350):
        try:
            mid = future_to_mids[future]
            transformed.append(transform_performance_for_evaluations(future.result(), mid,  2017))
        except KeyError:
            errors.append(future_to_mids[future])

In [45]:
for t in transformed:
    load_evaluation(t)

In [46]:
transformed = []
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    future_to_mids = {executor.submit(extract_performance_data, mid, 2018): mid for mid in ids}
    errors = []
    for future in tqdm(concurrent.futures.as_completed(future_to_mids), total=350):
        try:
            mid = future_to_mids[future]
            transformed.append(transform_performance_for_evaluations(future.result(), mid,  2018))
        except KeyError:
            errors.append(future_to_mids[future])

In [47]:
for t in transformed:
    load_evaluation(t)